## Imports

In [ ]:
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from pathlib import Path

## Settings

In [ ]:
path_Q_mes = Path(r"\\hunzikerwater.ch\DFSHBT\Daten-Winterthur\Projekte\3000-"
    r"\3033\3033.11 GEP\05 Berechnungen Grundlagen\Messkampagne\Messungen"
    r"\Messdaten Messkampagne komplett\03 Auswertung\Messstellen.csv")

path_Q_m9 = Path(r"\\hunzikerwater.ch\DFSHBT\Daten-Winterthur\Projekte\3000-"
    r"\3033\3033.11 GEP\05 Berechnungen Grundlagen\Messkampagne\Messungen"
    r"\Messdaten Messkampagne komplett\03 Auswertung\MS9.csv")

path_Q_sim = Path(r"\\hunzikerwater.ch\DFSHBT\Daten-Winterthur\Projekte\3000-"
    r"\3033\3033.11 GEP\05 Berechnungen Grundlagen\Messkampagne\Messungen"
    r"\Messdaten Messkampagne komplett\03 Auswertung"
    r"\ResultateMike4R_230117.xlsx")

path_rain = Path(r"\\hunzikerwater.ch\DFSHBT\Daten-Winterthur\Projekte\3000-"
    r"\3033\3033.11 GEP\05 Berechnungen Grundlagen\Messkampagne\Messungen"
    r"\Messdaten Messkampagne komplett\02 Daten Aufbereitet"
    r"\4Regenereignisse_MikeImport_5min_mbo.xlsx")

save_dir = Path(r"\\hunzikerwater.ch\DFSHBT\Daten-Winterthur\Projekte\3000-"
    r"\3033\3033.11 GEP\05 Berechnungen Grundlagen\Messkampagne\Messungen"
    r"\Messdaten Messkampagne komplett\03 Auswertung")

## Read data

### Rain

In [ ]:
rain = pd.read_excel(path_rain)
rain.set_index(keys=rain.columns[2], inplace=True)

### Discharge

In [ ]:
date_format = '%d.%m.%Y %H:%M'
date_parser = lambda x: pd.to_datetime(x, format=date_format)

Q_mes = pd.read_csv(path_Q_mes, header=0, sep=';', parse_dates=[0],
    date_parser=date_parser, index_col=0)

In [ ]:
Q_m5 = Q_mes['M5']
Q_m6 = Q_mes['M6']
Q_m101 = Q_mes['M10.1']
Q_m102 = Q_mes['M10.2']
Q_m9 = Q_mes['M9']
Q_m2 = Q_mes['M2']

In [ ]:
date_format = '%d.%m.%Y %H:%M'
date_parser = lambda x: pd.to_datetime(x, format=date_format)

Q_t = pd.read_csv(path_Q_m9, header=0, sep=';', parse_dates=[0],
    date_parser=date_parser, index_col=0)

In [ ]:
Q_m9 = pd.concat((Q_m9, Q_t['M9']), axis='index')
Q_m9.sort_index(inplace=True)

In [ ]:
Q_sim = pd.read_excel(path_Q_sim, sheet_name='Discharge', header=0,
    skiprows=[1], usecols=list(range(0,8)))
Q_sim.drop(columns=[Q_sim.columns[0]], inplace=True)
Q_sim.set_index(keys=Q_sim.columns[0], inplace=True)
Q_sim = Q_sim * 1e3 # convert m3/s -> l/s

## Plots

In [ ]:
events = rain['Nr. Ereignis'].unique()
m_points = Q_mes.columns
mes = [Q_m5, Q_m6, Q_m101, Q_m102, Q_m9, Q_m2]
plt_col = len(events)
date_formatter = mdates.DateFormatter('%d.%m %H:%M')

fig, axs = plt.subplots(len(m_points)+1, plt_col)
for i, ev in enumerate(events):
    print(f"Rain event {ev}")
    rain_sub = rain[rain['Nr. Ereignis']==ev]
    rain_start = rain_sub.index[0].to_datetime64()
    print(f"Rain start: {pd.to_datetime(rain_start)}")
    rain_end = rain_sub.index[-1].to_datetime64()
    print(f"Rain end: {pd.to_datetime(rain_end)}")

    mask_df = lambda df: df.iloc[(df.index>=rain_start) &
        (df.index<=rain_end),:]
    Q_sim1 = mask_df(Q_sim)

    # width = 1 / rain_sub.shape[0]
    axs[0,i].plot(rain_sub.index, rain_sub['Niederschlag mm/5min'],
        label='Niederschlag', color='b')
    axs[0,i].legend()
    axs[0,i].set_title(f'Regenereignis {ev}')
    axs[0,i].set_xlabel('Zeit')
    axs[0,i].set_ylabel('Regenmenge [mm/5min]')
    axs[0,i].xaxis.set_major_formatter(date_formatter)
    for label in axs[0,i].get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')

    for j, (m, mp) in enumerate(zip(mes, m_points)):
        m2 = m.dropna()
        m3 = m2[(m2.index>=rain_start) & (m2.index<=rain_end)]

        Q_sim2 = Q_sim1[mp]
        Q_sim2 = Q_sim2.dropna()

        axs[j+1,i].plot(m3.index, m3, label=f'Messung {mp}')
        axs[j+1,i].plot(Q_sim2.index, Q_sim2.iloc[:], label=f'Simulation {mp}')
        axs[j+1,i].legend()
        axs[j+1,i].set_xlabel('Zeit')
        axs[j+1,i].set_ylabel('Durchfluss [l/s]')
        axs[j+1,i].xaxis.set_major_formatter(date_formatter)
        for label in axs[j+1,i].get_xticklabels(which='major'):
            label.set(rotation=30, horizontalalignment='right')
    
    print(20*"-")

fig.set_size_inches(40, 50)
save_path = save_dir / 'Plot_Gegenüberstellung_Messung-Mike2_230117.pdf'
fig.savefig(save_path, bbox_inches='tight')
plt.close()